# $$Databases$$


---

## Relational algebra

---

**Relational Algebra** is a formal language. It's an algebra that forms the underpinnings of implemented languages like `SQL`.

Three basic operators in relational algebra are:

* [Select](#select) $\sigma$
* [Project](#project) $\Pi$
* [Join](#join) $\bowtie$

A relation in relational algebra is basically a table.

Imagine we have several tables.

College table:

| cName | state   | enr   |
|------|------|------|
|------|------|------|
|------|------|------|
|------|------|------|

Student table:

| sID | sName   | GPA   | HS   |
|------|------|------|------|
|------|------|------|------|
|------|------|------|------|
|------|------|------|------|

And Apply table:

| sID | cName   | major   | dec   |
|------|------|------|------|
|------|------|------|------|
|------|------|------|------|
|------|------|------|------|

To get a table we need to only write the relation's name:

$$\text{Student}$$

<a name="select"></a>Here's how to *select* **rows** from Student table with GPA > 3.7:

$$\sigma_{\text{gpa}>3.7}\text{ Student}$$

or with GPA > 3.7 AND HS (high school size) < 1000:

$$\sigma_{\text{gpa}>3.7\wedge\text{HS}<1000}\text{ Student}$$

So the *select* ($\sigma$) operator picks certain **rows**.

<a name="project"></a>To pick **columns** we use the *project* ($\Pi$) operator. Typical use case:

$$\Pi_{\text{ column_name_1, column_name_2, ... , column_name_n}}\text{Relation}$$

We can combine operators:

$$\Pi_{\text{ column_name_1, column_name_2, ... , column_name_n}}(\sigma_{\text{gpa}>3.7\wedge\text{HS}<1000}\text{ Student})$$

How to "glue" tables so we'll have content from both? We use *cross-product* operator ($\times$), which gives us the cartesian product of both table. Main problem of such product are the duplicates. We can avoid this using the *select* operator. Here's an example:

$$\sigma_{\text{ Student.sID=Apply.sID }\land\text{ HS>1000 }\land\text{ major='CS' }\land\text{ dec='R' }}\text{(Student}\times\text{Apply)}$$

<a name="join"></a>Doing so we'll have a relation columns with same data (`Student.sID` and `Apply.sID`). We can solve it using *natural join* ($\bowtie$).

Natural join enforces equality on all attributes with same name and eliminates one copy of duplicate attributes.

Here's how to select names and GPAs of students with HS>1000 who applied to CS and were rejected:

$$\Pi_{\text{SName,GPA}}(\sigma_{\text{HS}>1000\wedge\text{major}=\text{'CS'}\wedge\text{dec}=\text{'R'}}(\text{Student}\bowtie\text{Apply}))$$

Natural join automatically sets values equal when attribute names are the same and then it removes the duplicate columns.

You can also use *theta join* operator, which adds a condition to the natural join:

$$\text{Exp}_{1}\bowtie_{\theta}\text{Exp}_{2}=\sigma_{\theta}(\text{Exp}_{1}\times\text{Exp}_{2})$$

To sum up:

Query (expression) on a set of relations (tables) produces relation (table) as a result.

* Simplest query: **relation name**
* Use **operators** to filtes, slice, combine
* Operators so far: **select** ($\sigma$), **project** ($\Pi$), **cross-product** ($\times$), **natural join** ($\bowtie$), **theta join** ($\bowtie_{\theta}$)

Now, let's learn about **set** operators, [**union**](#union), [**difference**](#difference) and [**intersection**](#intersection), the [**renaming**](#rename) operator, and different **notations** for expressions in relational algebra.

<a name="union"></a>**Union** operator ($\cup$) is used to form a list of values. Technically, in order to union two lists they have to have the same schema (same attribute name), but more on that later. Here's an example of a union of lists with different attibute name (getting a list of college and student names):

$$\Pi_{\text{CName}}\text{College}\cup\Pi_{\text{SName}}\text{Student}$$

<a name="difference"></a>**Difference** operator. Here's how to select IDs of students who didn't apply anywhere:

$$\Pi_{\text{SID}}\text{Student}-\Pi_{\text{SID}}\text{Apply}$$

And here's how to add other attributes to the difference:

$$\Pi_{\text{SName}}((\Pi_{\text{SID}}\text{Student}-\Pi_{\text{SID}}\text{Apply})\bowtie\text{Student})$$

<a name="difference"></a>**Intersection** operator. Relation to select names that are both a college name and a student name:

$$\Pi_{\text{CName}}\text{College}\cap\Pi_{\text{SName}}\text{Student}$$

<a name="rename"></a>**Rename** operator ($\rho$) is necessary to express certain queries in relational algebra. Here's how to use it to change the relation name ($R$) or attribute names ($A_n$):

* to change both: $\rho_{\text{R(A}_{1}\text{,A}_{2}\text{,...,A}_{\text{n}}\text{)}}\text{(Expression)}$ <- the general one
* to change only the relation name: $\rho_{\text{R}}\text{(Expression)}$
* to change only the attributes name: $\rho_{\text{A}_{1}\text{,A}_{2}\text{,...,A}_{\text{n}}}\text{(Expression)}$

You'll need the operator:

* to unify schemas for set operators. Example:

$$\rho_{\text{c(name)}}(\Pi_{\text{CName}}\text{College})\cup\rho_{\text{c(name)}}(\Pi_{\text{SName}}\text{Student})$$

* for disambiguation in "self-joins". Example, query to select pairs of colleges in same state:

$$\sigma_{n_{1}<n_{2}}(\rho_{c_{1}(n_{1},s,e_{1})}(\text{College})\bowtie\rho_{c_{2}(n_{2},s,e_{2})}(\text{College}))$$

Sometimes people use alternative notation, such as *assignment statements* and *expression trees*.

* assignment statements, pairs of colleges in same state:

$
C1:= \rho_{c1,s,e1}College
\\
C2:= \rho_{c2,s,e2}College
\\
CP := C1\bowtie C2
\\
Ans := \sigma_{n1<n2}CP
$

* expression tree, GPAs of students applying to CS in CA:

![expression tree](https://sun9-33.userapi.com/c857016/v857016945/1308f0/DkGVojNlBr8.jpg "expression tree")

---

## Relational design theory

---

* 1.[Overview](#overview)
* [temp](#decomposition)
* [temp](#uwu)

### 1. Overview
<a name="overview"></a>
How to design a database schema?

* Usually many designs possible
* Some are (much) better that others!
* How do we choose?

To build a nice schema we need to avoid **anomalies**. They are:

* Redundancy (e.g. name appears near the unique id in every row)
* Update (updates facts differently, e.g. updates some names near unique id but not every name)
* Deletion (deletes unique data because of one attribute)

Example of such relation:

Apply relation (SSN - social security number, unique):

`|SSN|sName|cName|HS|hobby|
|123|Ann|Stanford|PAHS|tennis|
|123|Ann|Berkley|PAHS|tennis|
|123|Ann|Berkley|PAHS|trumpet|`


How to avoid them? **Design by decomposition!**

The steps are:

1. Start with "mega" relations containing everything
2. Decompose into smaller, better relations with same info
3. Can do decomposition automatically

**Automatic decomposition**:

* "Mega" relations + properties of the data
* System decomposes based on properties
* Final set of relations satisfies *normal form* (no anomalies, no lost information)

**Properties and Normal Forms**:

* Functional dependencies $\implies$ Boyce-Codd Normal Form
    * \+ Multivalued dependencies $\implies$ Fourth Normal Form
    
**Functional Dependencies and BCNF**

Example: relation `Apply`(`SSN`, `sName`, `cName`)

* Redundancy; update and deletion anomalies
* Storing `SSN-sName` pair once for each college

There's *functional dependency* SSN $\rightarrow$ sName:
* Same `SSN` always has same `sName`
* Should store each `SSN`'s `sName` only once

**Boyce-Codd Normal Form**: *if `A` $\rightarrow$ `B` then `A` is a key

That way in Boyce-Codd Normal Form `SSN` has to be a key (unique). So the original `Apply` relation doesn't comply with Boyce-Codd Normal Form (same student `SSN` can apply to different colleges `cName`) and has to be decomposed into relations `Student`(`SSN`,`sName`) (`SSN` is a key now and the relation has a functional dependency `SSN` $\rightarrow$ `sName`) and `Apply`(`SSN`,`cName`) (this relation doesn't have a functional dependency).

**Multivalued Dependencied and 4NF**

Example: relation `Apply`(`SSN`,`cName`,`HS`)

* Redundancy; update and deletion anomalies
* Multiplicative effect (C colleges, H high schools will constitute C * H tuples (C+H preferred))
* Not addressed by BCNF: No functional dependencies (basically it follows BCNF)

Multivalued dependency (`SSN` $\twoheadrightarrow$ `cName`) $\land$ (`SSN` $\twoheadrightarrow$ `HS`)

* Given `SSN` has every combination of `cName` with `HS`
* Should store each `cName` and each `HS` for an `SSN` once

**Fourth Normal Form**: *if `A` $\twoheadrightarrow$ `B` then `A` is a key

We'll need to decompose the original relation into: `Apply`(`SSN`,`cName`) and `HighSchool`(`SSN`,`HS`). Thus we'll have C + H tupples instead of C * H.